### Preprocessing with Cutoff Date

In [1]:
import pandas as pd

In [2]:
gstore = pd.read_csv("/Users/aoluwolerotimi/Datasets/train_dejsonified.csv")

/var/folders/z1/1zds15w97_g7_fhs4_83dc3m0000gn/T/ipykernel_46188/3136096564.py:1: DtypeWarning: Columns (2,54) have mixed types. Specify dtype option on import or set low_memory=False.
  gstore = pd.read_csv("/Users/aoluwolerotimi/Datasets/train_dejsonified.csv")


In [3]:
# gstore.dtypes

In [4]:
gstore['date'] = pd.to_datetime(gstore['date'], format='%Y%m%d').dt.strftime('%Y-%m-%d')

In [5]:
# 6-month split is end of Feb 2017
print("Earliest Date:", (gstore['date']).min())
print("Latest Date:", (gstore['date']).max())

Earliest Date: 2016-08-01
Latest Date: 2017-08-01


In [ ]:
# beginning of august to end of Feb 
# beginning of march to beginning of august

In [6]:
cutoff_date = pd.to_datetime('2017-02-28')

In [7]:

gstore['date'] = pd.to_datetime(gstore['date'])

In [8]:
# Verify the updated data type
print(gstore['date'].dtype)

datetime64[ns]


In [9]:
df1 = gstore.loc[gstore['date'] <= cutoff_date]
df2 = gstore.loc[gstore['date'] > cutoff_date]


In [10]:
print(df1.shape[0] + df2.shape[0])

903653


In [11]:
# data collection cutoff dates
df1cutoff = pd.to_datetime('2016-11-30')
df2cutoff = pd.to_datetime('2017-06-30')

In [12]:
# set aside collection period and performance period data
df1p = df1.loc[df1['date'] > df1cutoff]

In [24]:
df1c = df1.loc[df1['date'] <= df1cutoff]

In [14]:
df1trainids = df1c['fullVisitorId'].unique().tolist()


In [15]:
import gc

In [23]:
df1train = pd.DataFrame(df1trainids, columns=['fullVisitorId'])

In [20]:
gstore['medium'].value_counts()
# could investigate cpc later for uplift

organic      381561
referral     330955
(none)       143026
cpc           25326
affiliate     16403
cpm            6262
(not set)       120
Name: medium, dtype: int64

In [21]:
gstore['campaign'].value_counts()

(not set)                                          865347
Data Share Promo                                    16403
AW - Dynamic Search Ads Whole Site                  14244
AW - Accessories                                     7070
test-liyuhz                                           392
AW - Electronics                                       96
Retail (DO NOT EDIT owners nophakun and tianyu)        50
AW - Apparel                                           46
All Products                                            4
Data Share                                              1
Name: campaign, dtype: int64

In [22]:
gstore['campaignCode'].value_counts()

11251kjhkvahf    1
Name: campaignCode, dtype: int64

In [25]:
from sklearn.preprocessing import OneHotEncoder

In [26]:
# Step 1: Create the OneHotEncoder instance
encoder = OneHotEncoder(sparse=False)

# Step 2: Fit and transform the data
channel_encoded = encoder.fit_transform(df1c[['channelGrouping']])

# Step 3: Convert the numpy array back to a DataFrame
channel_encoded_df = pd.DataFrame(channel_encoded, columns=encoder.get_feature_names_out(['channelGrouping']))

# Step 4: Concatenate the new dataframe with the original dataframe minus the original 'channelGrouping' column
df1c = pd.concat([df1c.drop('channelGrouping', axis=1), channel_encoded_df], axis=1)

/Users/aoluwolerotimi/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [28]:
# df1c.columns

In [29]:
cg = ['channelGrouping_(Other)', 'channelGrouping_Affiliates',
       'channelGrouping_Direct', 'channelGrouping_Display',
       'channelGrouping_Organic Search', 'channelGrouping_Paid Search',
       'channelGrouping_Referral', 'channelGrouping_Social']

In [32]:
df1train = df1train[['fullVisitorId']]

In [30]:
# for column in cg:
#     df1train[column] = pd.NA

In [33]:
# Step 1: Group by 'fullVisitorId' in df1c and sum the columns in cg
grouped = df1c.groupby('fullVisitorId')[cg].sum()

# Step 2: Merge this grouped data back into df1train
df1train = df1train.merge(grouped, on='fullVisitorId', how='left')

              fullVisitorId  channelGrouping_(Other)  \
0       1131660440785968503                      0.0   
1        377306020877927890                      0.0   
2       3895546263509774583                      0.0   
3       4763447161404445595                      0.0   
4         27294437909732085                      0.0   
...                     ...                      ...   
299069  3118235170138318723                      0.0   
299070  2900750270670203622                      0.0   
299071  4821618326715417613                      0.0   
299072  2720938310295703834                      0.0   
299073   593617143165950851                      0.0   

        channelGrouping_Affiliates  channelGrouping_Direct  \
0                              0.0                     0.0   
1                              0.0                     0.0   
2                              0.0                     0.0   
3                              0.0                     0.0   
4                

In [34]:
# Step 1: Count occurrences of each 'fullVisitorId' in df1c
visitor_counts = df1c['fullVisitorId'].value_counts().reset_index()
visitor_counts.columns = ['fullVisitorId', 'totalVisits']

# Step 2: Merge this count DataFrame back into df1train
df1train = df1train.merge(visitor_counts, on='fullVisitorId', how='left')

In [35]:
# continue later
# df1c.to_csv('df1c.csv', index=True)
# df1train.to_csv('df1train.csv', index=True)

* Channel grouping dummies then sum --> DONE
* Count appearances of vistor id for total visits in period
* First Continent, Suboncontinent, Country appearing
* Device category dummies then summed
* Sum of hits
* Sum of pageviews
* Flag for if new visitor in the period
* Sum of transaction value in the period (might need to set nan to 0 first)
* Source dummies then summed then PCA
* Sum isTrueDirect
* adWordsgclID sum not null 
* Sum campaign not null (set not set to null)